# Presentation Notebook

## Import library

In [1]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv
from cassandra.cluster import Cluster
from prettytable import PrettyTable

## Create the function for process the multiple csv files (raw data) into one csv file

This function,
1. open all csv files and read
2. extract the data
3. create csv file accroding to a table which has columns artist name, user firstname, gender,item in session, user lastname, song length, level, location, session id, song name, and user id

    """
    Description: This function read all the csv file from event_data folder into event_datafile_new.csv
    
    Arguments:
        path: folder name
        
    Returns:
        None
    """

In [2]:
def process_csv(path):
    filepath = os.getcwd() + path

    for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root,'*'))
    full_data_rows_list = [] 
    for f in file_path_list:

        with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
            csvreader = csv.reader(csvfile) 
            next(csvreader)    
            for line in csvreader:
                full_data_rows_list.append(line) 

    csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

    with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
        writer = csv.writer(f, dialect='myDialect')
        writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                    'level','location','sessionId','song','userId'])
        for row in full_data_rows_list:
            if (row[0] == ''):
                continue
            writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16])) 

## Create the function for checking the row in csv file

In this function, purpose is to check that the data exists in the file or not

    """
    Description: This function for checking the row in csv file
    
    Arguments:
        file: csv file name
        
    Returns:
        None
    """

In [3]:
def check_numrow(file):
    with open(file, 'r', encoding = 'utf8') as f:
        print(sum(1 for line in f))

## Create the function for create keyspace

In this function, do 2 process as follows:
1. Create Keyspace
2. Set the Keyspace

    """
    Description: This function create the keyspace for Cassandra using the specific name user provide
   
    Arguments:
        name: the name for keyspace
        
    Returns:
        None
    """

In [4]:
def create_keyspace(name):
    query = "CREATE KEYSPACE IF NOT EXISTS " + name +" WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"
    try:
        session.execute(query)
    except Exception as e:
        print(e)
    try:
        session.set_keyspace(name)
    except Exception as e:
        print(e)

## Create the fuction for create table and push all data

In this function,
1. will drop the table if the table already exists in the system if not it will pass to the next process
2. Create the table according to the 'table' argument with 'primary' as the PRIMARY KEY 
3. Then it will INSERT the data into the table and the data come from 'event_datafile_new.csv'

    """
    Description: This function create the table according to the name and Primary key
    
    Arguments:
        table: the name for table
        primary: the primary key for table
        opt: the option must be select 1-3 according to the SELECT query
        
    Returns:
        None
    """

In [5]:
def create_data(table, primary, opt):
    head = "DROP TABLE IF EXISTS " + table
    session.execute(head)        
    query = "CREATE TABLE IF NOT EXISTS " + table
    if opt==1:
        query = query + "(sessionid text, iteminsession text, artist_name text, song_title text,song_length text, user_firstname text, user_lastname text, userid text,\
        PRIMARY KEY ("+primary+" ))"
    elif opt==2:
        query = query + "(sessionid text, userid text, iteminsession text, artist_name text, song_title text,song_length text, user_firstname text, user_lastname text, \
        PRIMARY KEY ("+primary+" ))"
    elif opt==3:
        query = query + "(song_title text,user_firstname text, user_lastname text, userid text,artist_name text, song_length text, sessionid text, iteminsession text,  \
        PRIMARY KEY ("+primary+" ))"

    try:
        session.execute(query)
        insert_data(table,opt)
    except Exception as e:
        print(e)       

    """
    Description: This function insert all data from .csv file into the table
    
    Arguments:
        table: the name for table
        opt: the option must be select 1-3
        
    Returns:
        None
    """

In [6]:
def insert_data(table, opt):
    file = 'event_datafile_new.csv'
    with open(file, encoding = 'utf8') as f:
        csvreader = csv.reader(f)
        next(csvreader)
        if opt==1:
            for line in csvreader:
                query = "INSERT INTO "+table+"(sessionid,iteminsession,artist_name,song_title,song_length,user_firstname,user_lastname,userid)"
                query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
                data_line = (line[-3],line[3],line[0],line[-2],line[5],line[1],line[4],line[-1])
                session.execute(query, data_line)
        elif opt==2:
            for line in csvreader:
                query = "INSERT INTO "+table+"(sessionid,userid,iteminsession,artist_name,song_title,song_length,user_firstname,user_lastname)"
                query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
                data_line = (line[-3],line[-1],line[3],line[0],line[-2],line[5],line[1],line[4])
                session.execute(query, data_line)
        elif opt==3:
             for line in csvreader:
                query = "INSERT INTO "+table+"(song_title,user_firstname,user_lastname,userid,artist_name,song_length,sessionid,iteminsession )"
                query = query + "VALUES(%s,%s,%s,%s,%s,%s,%s,%s)"
                data_line = (line[-2],line[1],line[4],line[-1],line[0],line[5],line[-3],line[3])
                session.execute(query, data_line)

## Create the function for select the data according to the query 1-3

In this function create for select the data according to the primary and clustering key specify in this project which have 3 selections.

1. Print artistist name, song name, and song length
2. Print artistist name, song name, user firstname, and user lastname
3. Print user firstname and user lastname

And for the select query we need to specify statement (fwh) for those 3 selections as well

    """
    Description: This function select the data according to the table and query
    
    Arguments:
        table: the name for table
        fwh: mean From the WHERE statement (From WHere)
        opt: option for select
        select_field: for select the specific column in SELECT instead of 'SELECT * ' (for cost sufficient)
        
    Returns:
        None
    """

In [7]:
def select_data(table, fwh, opt, select_field) :
    query = "select "+select_field+" from "+table+" WHERE "+fwh 
    try:
        rows = session.execute(query)
    except Exception as e:
        print(e)
    if opt==1:
        col = PrettyTable(['Artist', 'Song', 'Length'])
        for row in rows:
            col.add_row([row.artist_name, row.song_title, row.song_length])
    elif opt==2:
        col = PrettyTable(['Artist', 'Song', 'User Firstname', 'User Lastname'])
        for row in rows:
            col.add_row([row.artist_name, row.song_title, row.user_firstname, row.user_lastname])
    elif opt==3:
        col = PrettyTable(['User Firstname', 'User Lastname'])
        for row in rows:
            col.add_row([row.user_firstname, row.user_lastname])
    else:
        print('error')
    print(col)

## Perform all the function in order


1. start the connection in the name of session
2. The start process the csv file
3. Create KeySpace
4. Create and Insert table for song_playlist 1,song_playlist 2,song_playlist 3
5. Select the data from song_playlist 1
6. Select the data from song_playlist 2
7. Select the data from song_playlist 3

In [11]:
cluster = Cluster()
session = cluster.connect()
process_csv('/event_data')
create_keyspace('cassandrapj')
print('Preparing Data....')
create_data('song_playlist_session', 'sessionid, iteminsession',1)
create_data('song_playlist_user', '(sessionid, userid), iteminsession',2)
create_data('song_playlist_songtitle', '(song_title), user_firstname,user_lastname, userid',3)
print('Song Playlist 1st Table')
select_data('song_playlist_session', "sessionid='338' and iteminsession='4'",1, "artist_name,song_title,song_length")
print('Song Playlist 2nd Table')
select_data('song_playlist_user',"sessionid='182' AND userid='10'",2, "artist_name,song_title,user_firstname,user_lastname")
print('Song Playlist 3rd Table')
select_data('song_playlist_songtitle',"song_title='All Hands Against His Own'",3,"user_firstname,user_lastname")

Preparing Data....
Song Playlist 1st Table
+-----------+---------------------------------+----------+
|   Artist  |               Song              |  Length  |
+-----------+---------------------------------+----------+
| Faithless | Music Matters (Mark Knight Dub) | 495.3073 |
+-----------+---------------------------------+----------+
Song Playlist 2nd Table
+-------------------+------------------------------------------------------+----------------+---------------+
|       Artist      |                         Song                         | User Firstname | User Lastname |
+-------------------+------------------------------------------------------+----------------+---------------+
|  Down To The Bone |                  Keep On Keepin' On                  |     Sylvie     |      Cruz     |
|    Three Drives   |                     Greece 2000                      |     Sylvie     |      Cruz     |
| Sebastien Tellier |                      Kilometer                       |     Sylvie 